In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers datasets evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dc8f5da623996ea459257d32975724ebf4b1c3ac289c2e2158dc3b47e01b4f4c
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

In [ ]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")
billsum = billsum.train_test_split(test_size=0.2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 399.13 of the Public Utilities Code is amended to read:\n399.13.\n(a) (1) The commission shall direct each electrical corporation to annually prepare a renewable energy procurement plan that includes the matter in paragraph (5), to satisfy its obligations under the renewables portfolio standard. To the extent feasible, this procurement plan shall be proposed, reviewed, and adopted by the commission as part of, and pursuant to,\na\nthe\ngeneral procurement plan\nprocess.\nprocess (Sections 454.5 to 454.55, inclusive).\nThe commission shall require each electrical corporation to review and update its renewable energy procurement plan as it determines to be necessary. The commission shall require all other retail sellers to prepare and submit renewable energy procurement plans that address the requirements identified in paragraph (5).\n(2) Every electrical corporation that owns electrical transmi

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [ ]:
tokenized_billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 248
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)


In [ ]:
import evaluate
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)






config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Summerize/Epoch6",
    eval_strategy="epoch",
    logging_dir="/content/drive/MyDrive/Summerize/Result",
    logging_steps=50,
    learning_rate=2e-5,
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    report_to=["none"],
)


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-21-fe02c4190ceb>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import numpy as np


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.561600,2.345640,0.203000,0.101800,0.172700,0.173000,20.000000
2,2.567500,2.315654,0.204900,0.103600,0.173000,0.173300,20.000000
3,2.468100,2.292059,0.204600,0.103900,0.173800,0.174100,20.000000
4,2.502000,2.280315,0.205800,0.104800,0.174300,0.174700,20.000000
5,2.459600,2.273489,0.205100,0.103900,0.173600,0.173800,20.000000
6,2.450400,2.271614,0.204900,0.104000,0.173600,0.173900,20.000000


TrainOutput(global_step=372, training_loss=2.4970081698509956, metrics={'train_runtime': 427.3565, 'train_samples_per_second': 13.885, 'train_steps_per_second': 0.87, 'total_flos': 1606236499869696.0, 'train_loss': 2.4970081698509956, 'epoch': 6.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Summerize/Upload")
tokenizer.save_pretrained("p/content/drive/MyDrive/Summerize/Upload")

('p/content/drive/MyDrive/Summerize/Upload/tokenizer_config.json',
 'p/content/drive/MyDrive/Summerize/Upload/special_tokens_map.json',
 'p/content/drive/MyDrive/Summerize/Upload/spiece.model',
 'p/content/drive/MyDrive/Summerize/Upload/added_tokens.json',
 'p/content/drive/MyDrive/Summerize/Upload/tokenizer.json')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Summerize/Epoch6/checkpoint-372")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Summerize/Epoch6/checkpoint-372")

# Sample input text (for summarization, this could be a paragraph or article)
input_text = """
The quick brown fox jumps over the lazy dog. This sentence is often used to test typing and font rendering.
It contains all the letters of the English alphabet, making it a pangram. This sentence has been widely used
for centuries and is still relevant in the digital age. In fact, it remains a popular sentence in the world of
typography.
"""

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Generate output (e.g., summary)
summary_ids = model.generate(
    inputs['input_ids'],  # Input tokens
    max_length=150,  # Maximum length of the output sequence
    num_beams=4,  # Beam search to improve output quality
    no_repeat_ngram_size=2,  # Avoid repeating n-grams
    length_penalty=2.0,  # Control length of output
    early_stopping=True  # Stop as soon as we reach the maximum length
)

# Decode and print the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


This sentence is often used to test typing and font rendering. It contains all the letters of the English alphabet, making it a pangram. The quick brown fox jumps over the lazy dog.


In [ ]:
import os

# Check the contents of the model directory
model_dir = "/content/drive/MyDrive/Summerize/Epoch6"
print(os.listdir(model_dir))


['checkpoint-372']
